In [1]:
# check gpu
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16928081446201156588
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5750390784
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16237647787032238677
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [139]:
# 실제 train data를 변환하기 - 데이터 읽어들이기

import json
import os
import pandas as pd
import numpy as np
import shutil
import zipfile

base_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터'

train_images_path = base_dir + '/train/images/'
train_labels_json_path = base_dir + '/train/labels_json/'
train_labels_path = base_dir + '/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

img_zip_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/Training/01.원천데이터/'
img_zip_name = 'TS_1.영상추출_07.유리병_003.투명'
# shutil.unpack_archive(img_zip_dir + img_zip_name + '.zip', img_zip_dir + img_zip_name, 'zip')
zipfile_img = zipfile.ZipFile(img_zip_dir + img_zip_name + '.zip')
zipfile_img.extractall(path=img_zip_dir + img_zip_name)

label_zip_dir = 'D:/python-ai-data/232.재활용품 분류 및 선별 데이터/01.데이터/Training/02.라벨링데이터/'
label_zip_name = 'TL_1.영상추출_07.유리병_003.투명'
# shutil.unpack_archive(label_zip_dir + label_zip_name + '.zip', label_zip_dir + label_zip_name, 'zip')
zipfile_label = zipfile.ZipFile(label_zip_dir + label_zip_name + '.zip')
zipfile_label.extractall(path=label_zip_dir + label_zip_name)

zipfile_img.close()
zipfile_label.close()

# !unzip -qq C:/python-project/AI/train.zip -d C:/python-project/AI/train2/


In [140]:

train_images_list = os.listdir(img_zip_dir + img_zip_name)
train_labels_json_list = os.listdir(label_zip_dir + label_zip_name)

train_images_list.sort()
train_labels_json_list.sort()

print(len(train_images_list))
print(len(train_labels_json_list))

7254
7254


In [143]:
# 실제 train data를 변환하기 - 라벨링 변환

yolo_class = {
    '철캔': 0,
    '알루미늄캔': 4,
    '종이': 8,
    '무색단일': 12,
    '유색단일': 16,
    'PE': 20,
    'PP': 24,
    'PS': 28,
    '스티로폼': 32,
    '비닐': 36,
    '갈색': 40,
    '녹색': 44,
    '투명': 48
}

labels_path0 = base_dir + '/data/' + label_zip_name + '/labels0/'
labels_path1 = base_dir + '/data/' + label_zip_name + '/labels1/'
labels_path2 = base_dir + '/data/' + label_zip_name + '/labels2/'
labels_path3 = base_dir + '/data/' + label_zip_name + '/labels3/'

images_path0 = base_dir + '/data/' + img_zip_name + '/images0/'
images_path1 = base_dir + '/data/' + img_zip_name + '/images1/'
images_path2 = base_dir + '/data/' + img_zip_name + '/images2/'
images_path3 = base_dir + '/data/' + img_zip_name + '/images3/'

os.makedirs(labels_path0, exist_ok=True)
os.makedirs(labels_path1, exist_ok=True)
os.makedirs(labels_path2, exist_ok=True)
os.makedirs(labels_path3, exist_ok=True)

os.makedirs(images_path0, exist_ok=True)
os.makedirs(images_path1, exist_ok=True)
os.makedirs(images_path2, exist_ok=True)
os.makedirs(images_path3, exist_ok=True)

import shutil
import os
from PIL import Image

img_resize = 640

i = 0
for image_name in train_images_list:
    label = image_name.replace('.jpg', '.json')
    file_json = open(label_zip_dir + label_zip_name + '/' + label, encoding='UTF-8')
    f_json = json.load(file_json)
    file_json.close()
    img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
    img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

    i += 1
    if i % 100 == 0:
        print(i)
        if min([len(os.listdir(images_path0)), len(os.listdir(images_path1)), 
                len(os.listdir(images_path2)), len(os.listdir(images_path3))]) > 500:
            break
        if max([len(os.listdir(images_path0)), len(os.listdir(images_path1)), 
                len(os.listdir(images_path2)), len(os.listdir(images_path3))]) > 3000:
            pass
#             break

    for anno in f_json['ANNOTATION_INFO']:
        obj_id = yolo_class[anno['DETAILS']]

        if anno['DAMAGE'] != '원형':
            obj_id += 2
        if anno['DIRTINESS'] != '오염없음':
            obj_id += 1
        
        image = label.replace('.json', '.jpg')
        
        try:
            if obj_id % 4 == 0:
#                 continue
                shutil.copy(label_zip_dir + label_zip_name + '/' + label, labels_path0 + label)
                image_ori = Image.open(img_zip_dir + img_zip_name + '/' + image)
                image_tra = image_ori.resize((img_resize,img_resize))
                image_tra.save(images_path0 + image)
                image_ori.close()
            elif obj_id % 4 == 1:
                continue
                shutil.copy(label_zip_dir + label_zip_name + '/' + label, labels_path1 + label)
                image_ori = Image.open(img_zip_dir + img_zip_name + '/' + image)
                image_tra = image_ori.resize((img_resize,img_resize))
                image_tra.save(images_path1 + image)
                image_ori.close()
            elif obj_id % 4 == 2:
#                 continue
                shutil.copy(label_zip_dir + label_zip_name + '/' + label, labels_path2 + label)
                image_ori = Image.open(img_zip_dir + img_zip_name + '/' + image)
                image_tra = image_ori.resize((img_resize,img_resize))
                image_tra.save(images_path2 + image)
                image_ori.close()
            else:
#                 continue
                shutil.copy(label_zip_dir + label_zip_name + '/' + label, labels_path3 + label)
                image_ori = Image.open(img_zip_dir + img_zip_name + '/' + image)
                image_tra = image_ori.resize((img_resize,img_resize))
                image_tra.save(images_path3 + image)
                image_ori.close()
        except:
            print(image)
print(i)
print('****************')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7254
****************


In [144]:
print(len(os.listdir(labels_path0)))
print(len(os.listdir(labels_path1)))
print(len(os.listdir(labels_path2)))
print(len(os.listdir(labels_path3)))

print(len(os.listdir(images_path0)))
print(len(os.listdir(images_path1)))
print(len(os.listdir(images_path2)))
print(len(os.listdir(images_path3)))

805
3003
135
1076
805
3003
135
1076


In [145]:
label0 = os.listdir(labels_path0)
label1 = os.listdir(labels_path1)
label2 = os.listdir(labels_path2)
label3 = os.listdir(labels_path3)

label0.sort()
label1.sort()
label2.sort()
label3.sort()

image0 = os.listdir(images_path0)
image1 = os.listdir(images_path1)
image2 = os.listdir(images_path2)
image3 = os.listdir(images_path3)

image0.sort()
image1.sort()
image2.sort()
image3.sort()

for label in label0:
    try:
        if image0.index(label.replace('.json', '.jpg')):
            pass
    except:
        print(label)

print('***********************')

for label in label1:
    try:
        if image1.index(label.replace('.json', '.jpg')):
            pass
    except:
        print(label)

print('***********************')

for label in label2:
    try:
        if image2.index(label.replace('.json', '.jpg')):
            pass
    except:
        print(label)

print('***********************')

for label in label3:
    try:
        if image3.index(label.replace('.json', '.jpg')):
            pass
    except:
        print(label)
        

***********************
***********************
***********************
